In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_.drop_duplicates(['JOIN_PNB'], keep='first').reset_index(drop=True)

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                resp = requests.get('https://www.drivparts.com/driv/partfinder/api.catalog.applications?brand=corporate&locale=en_US&country_code=US&part_number=' + input_.loc[a, 'Part_Number'] + '&brand_code=' + input_.loc[a, 'Brand'],
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_application = json_data['application_group_list']['PASSENGER CAR & LIGHT TRUCK']['applications']

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'JOIN_PNB': input_.loc[a, 'JOIN_PNB'],
                                        'Vehicle_No.': [i+1 for i in range(len(list_application))]})

                for i in range(len(list_application)):
                    for name in list_application[i]:
                        if name == 'category':
                            df_temp.loc[i, 'description'] = list_application[i][name]['part_type_value'].strip()
                        elif name == 'engine_base' or name == 'engine_version_value':
                            pass
                        elif type(list_application[i][name]) == dict:
                            df_temp.loc[i, name] = list_application[i][name]['value'].strip()
                        elif name == 'years':
                            year_min = min(list_application[i][name])
                            year_max = max(list_application[i][name])
                            if year_min == year_max:
                                df_temp.loc[i, name] = str(year_min)
                            else:
                                df_temp.loc[i, name] = str(year_min) + '-' + str(year_max)
                        elif type(list_application[i][name]) == int:
                            df_temp.loc[i, name] = list_application[i][name]
                        elif type(list_application[i][name]) == str:
                            df_temp.loc[i, name] = list_application[i][name].strip()
                        else:
                            print('?')

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Part_Number': input_.loc[a, 'Part_Number'],
                                     'Brand': input_.loc[a, 'Brand'],
                                     'JOIN_PNB': input_.loc[a, 'JOIN_PNB']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Part_Number'] + ' - ' + input_.loc[a, 'Brand'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(15):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_PNB', 'Vehicle_No.'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Part_Number', 'Brand'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1049

[ok] - 512555 - FLLJ
[尝试次数：1] - [剩余数量：1034] - [当前时间：15:48:47]

[ok] - 513446 - FLLJ
[尝试次数：1] - [剩余数量：1033] - [当前时间：15:48:47]

[ok] - 512646 - FLLJ
[尝试次数：1] - [剩余数量：1032] - [当前时间：15:48:47]

[ok] - 512665 - FLLJ
[尝试次数：1] - [剩余数量：1031] - [当前时间：15:48:47]

[ok] - 513410 - FLLJ
[尝试次数：1] - [剩余数量：1030] - [当前时间：15:48:47]

[ok] - 513324 - FLLJ
[尝试次数：1] - [剩余数量：1029] - [当前时间：15:48:47]

[ok] - 513225 - FLLJ
[尝试次数：1] - [剩余数量：1028] - [当前时间：15:48:47]

[ok] - 512576 - FLLJ
[尝试次数：1] - [剩余数量：1027] - [当前时间：15:48:47]

[ok] - 513420 - FLLJ
[尝试次数：1] - [剩余数量：1026] - [当前时间：15:48:47]

[ok] - 513451 - FLLJ
[尝试次数：1] - [剩余数量：1025] - [当前时间：15:48:47]

[ok] - 512513 - FLLJ
[尝试次数：1] - [剩余数量：1024] - [当前时间：15:48:47]

[ok] - 512517 - FLLJ
[尝试次数：1] - [剩余数量：1023] - [当前时间：15:48:48]

[ok] - 513452 - FLLJ
[尝试次数：1] - [剩余数量：1022] - [当前时间：15:48:48]

[ok] - 515207 - FLLJ
[尝试次数：1] - [剩余数量：1021] - [当前时间：15:48:48]

[ok] - 513325 - FLLJ
[尝试次数：1] - [剩余数量：1020] - [当前时间：15:48:48]

[ok] - 512668 - FLLJ
[尝试次数：1] - [剩余数量：1019] -

[ok] - 512628 - FLLJ
[尝试次数：6] - [剩余数量：900] - [当前时间：15:49:06]

[ok] - 513354 - FLLJ
[尝试次数：10] - [剩余数量：899] - [当前时间：15:49:06]

[ok] - 512479 - FLLJ
[尝试次数：4] - [剩余数量：898] - [当前时间：15:49:06]

[ok] - 515188 - FLLJ
[尝试次数：1] - [剩余数量：897] - [当前时间：15:49:06]

[ok] - 512663 - FLLJ
[尝试次数：1] - [剩余数量：896] - [当前时间：15:49:06]

[ok] - 513288 - FLLJ
[尝试次数：3] - [剩余数量：895] - [当前时间：15:49:07]

[ok] - 512498 - FLLJ
[尝试次数：1] - [剩余数量：894] - [当前时间：15:49:07]

[ok] - 515146 - FLLJ
[尝试次数：4] - [剩余数量：893] - [当前时间：15:49:07]

[ok] - 512478 - FLLJ
[尝试次数：1] - [剩余数量：892] - [当前时间：15:49:07]

[ok] - 515176 - FLLJ
[尝试次数：5] - [剩余数量：891] - [当前时间：15:49:08]

[ok] - 512497 - FLLJ
[尝试次数：5] - [剩余数量：890] - [当前时间：15:49:08]

[ok] - 515160 - FLLJ
[尝试次数：6] - [剩余数量：889] - [当前时间：15:49:08]

[ok] - 512516 - FLLJ
[尝试次数：2] - [剩余数量：888] - [当前时间：15:49:09]

[ok] - 515191 - FLLJ
[尝试次数：1] - [剩余数量：887] - [当前时间：15:49:09]

[ok] - 513430 - FLLJ
[尝试次数：2] - [剩余数量：886] - [当前时间：15:49:09]

[ok] - 512666 - FLLJ
[尝试次数：4] - [剩余数量：885] - [当前时间：15:49:09]

[ok] - 

[ok] - 515065 - FLLJ
[尝试次数：24] - [剩余数量：767] - [当前时间：15:49:32]

[ok] - LK048 - FLLJ
[尝试次数：3] - [剩余数量：766] - [当前时间：15:49:32]

[ok] - LK047 - FLLJ
[尝试次数：5] - [剩余数量：765] - [当前时间：15:49:32]

[ok] - 512525 - FLLJ
[尝试次数：1] - [剩余数量：764] - [当前时间：15:49:33]

[ok] - 513257 - FLLJ
[尝试次数：4] - [剩余数量：763] - [当前时间：15:49:33]

[ok] - 512552 - FLLJ
[尝试次数：3] - [剩余数量：762] - [当前时间：15:49:33]

[ok] - 512437 - FLLJ
[尝试次数：2] - [剩余数量：761] - [当前时间：15:49:33]

[ok] - 513377 - FLLJ
[尝试次数：1] - [剩余数量：760] - [当前时间：15:49:34]

[ok] - 512483 - FLLJ
[尝试次数：1] - [剩余数量：759] - [当前时间：15:49:34]

[ok] - 512345 - FLLJ
[尝试次数：14] - [剩余数量：758] - [当前时间：15:49:34]

[ok] - 512524 - FLLJ
[尝试次数：2] - [剩余数量：757] - [当前时间：15:49:34]

[ok] - 512403 - FLLJ
[尝试次数：7] - [剩余数量：756] - [当前时间：15:49:35]

[ok] - 515192 - FLLJ
[尝试次数：1] - [剩余数量：755] - [当前时间：15:49:35]

[ok] - 512529 - FLLJ
[尝试次数：4] - [剩余数量：754] - [当前时间：15:49:35]

[ok] - 512417 - FLLJ
[尝试次数：6] - [剩余数量：753] - [当前时间：15:49:35]

[ok] - 512523 - FLLJ
[尝试次数：11] - [剩余数量：752] - [当前时间：15:49:36]

[ok] - 

[ok] - 515096 - FLLJ
[尝试次数：1] - [剩余数量：634] - [当前时间：15:49:59]

[ok] - 512549 - FLLJ
[尝试次数：1] - [剩余数量：633] - [当前时间：15:49:59]

[ok] - 512395 - FLLJ
[尝试次数：4] - [剩余数量：632] - [当前时间：15:50:00]

[ok] - 513392 - FLLJ
[尝试次数：5] - [剩余数量：631] - [当前时间：15:50:00]

[ok] - 515036 - FLLJ
[尝试次数：5] - [剩余数量：630] - [当前时间：15:50:01]

[ok] - 512369 - FLLJ
[尝试次数：2] - [剩余数量：629] - [当前时间：15:50:01]

[ok] - 515174 - FLLJ
[尝试次数：4] - [剩余数量：628] - [当前时间：15:50:01]

[ok] - 513326 - FLLJ
[尝试次数：3] - [剩余数量：627] - [当前时间：15:50:01]

[ok] - 515166 - FLLJ
[尝试次数：1] - [剩余数量：626] - [当前时间：15:50:01]

[ok] - 513224 - FLLJ
[尝试次数：8] - [剩余数量：625] - [当前时间：15:50:02]

[ok] - LK051 - FLLJ
[尝试次数：1] - [剩余数量：624] - [当前时间：15:50:02]

[ok] - 512391 - FLLJ
[尝试次数：2] - [剩余数量：623] - [当前时间：15:50:02]

[ok] - LK052 - FLLJ
[尝试次数：1] - [剩余数量：622] - [当前时间：15:50:02]

[ok] - LK057 - FLLJ
[尝试次数：4] - [剩余数量：621] - [当前时间：15:50:03]

[ok] - 512545 - FLLJ
[尝试次数：1] - [剩余数量：620] - [当前时间：15:50:03]

[ok] - 515142 - FLLJ
[尝试次数：10] - [剩余数量：619] - [当前时间：15:50:03]

[ok] - 512

[ok] - LK042 - FLLJ
[尝试次数：1] - [剩余数量：501] - [当前时间：15:50:26]

[ok] - 512289 - FLLJ
[尝试次数：3] - [剩余数量：500] - [当前时间：15:50:26]

[ok] - 518510 - FLLJ
[尝试次数：10] - [剩余数量：499] - [当前时间：15:50:26]

[ok] - 512250 - FLLJ
[尝试次数：12] - [剩余数量：498] - [当前时间：15:50:26]

[ok] - 515126 - FLLJ
[尝试次数：1] - [剩余数量：497] - [当前时间：15:50:27]

[ok] - 512426 - FLLJ
[尝试次数：5] - [剩余数量：496] - [当前时间：15:50:27]

[ok] - 515122 - FLLJ
[尝试次数：2] - [剩余数量：495] - [当前时间：15:50:27]

[ok] - 515123 - FLLJ
[尝试次数：1] - [剩余数量：494] - [当前时间：15:50:27]

[ok] - LK004 - FLLJ
[尝试次数：10] - [剩余数量：493] - [当前时间：15:50:27]

[ok] - 513230 - FLLJ
[尝试次数：15] - [剩余数量：492] - [当前时间：15:50:28]

[ok] - 512341 - FLLJ
[尝试次数：2] - [剩余数量：491] - [当前时间：15:50:28]

[ok] - 512410 - FLLJ
[尝试次数：1] - [剩余数量：490] - [当前时间：15:50:28]

[ok] - 513341 - FLLJ
[尝试次数：7] - [剩余数量：489] - [当前时间：15:50:28]

[ok] - 512435 - FLLJ
[尝试次数：1] - [剩余数量：488] - [当前时间：15:50:28]

[ok] - 541009 - FLLJ
[尝试次数：3] - [剩余数量：487] - [当前时间：15:50:28]

[ok] - 515129 - FLLJ
[尝试次数：5] - [剩余数量：486] - [当前时间：15:50:29]

[ok] -

[ok] - 515114 - FLLJ
[尝试次数：10] - [剩余数量：368] - [当前时间：15:50:53]

[ok] - 515109 - FLLJ
[尝试次数：4] - [剩余数量：367] - [当前时间：15:50:53]

[ok] - 512218 - FLLJ
[尝试次数：3] - [剩余数量：366] - [当前时间：15:50:53]

[ok] - 512188 - FLLJ
[尝试次数：4] - [剩余数量：365] - [当前时间：15:50:53]

[ok] - 518516 - FLLJ
[尝试次数：1] - [剩余数量：364] - [当前时间：15:50:54]

[ok] - 515079 - FLLJ
[尝试次数：5] - [剩余数量：363] - [当前时间：15:50:54]

[ok] - LK009 - FLLJ
[尝试次数：4] - [剩余数量：362] - [当前时间：15:50:54]

[ok] - 513237 - FLLJ
[尝试次数：9] - [剩余数量：361] - [当前时间：15:50:54]

[ok] - 512221 - FLLJ
[尝试次数：6] - [剩余数量：360] - [当前时间：15:50:54]

[ok] - 513157 - FLLJ
[尝试次数：2] - [剩余数量：359] - [当前时间：15:50:54]

[ok] - 513260 - FLLJ
[尝试次数：1] - [剩余数量：358] - [当前时间：15:50:55]

[ok] - LK071 - FLLJ
[尝试次数：10] - [剩余数量：357] - [当前时间：15:50:55]

[ok] - LK064 - FLLJ
[尝试次数：2] - [剩余数量：356] - [当前时间：15:50:55]

[ok] - LK029 - FLLJ
[尝试次数：4] - [剩余数量：355] - [当前时间：15:50:55]

[ok] - 512375 - FLLJ
[尝试次数：4] - [剩余数量：354] - [当前时间：15:50:55]

[ok] - 513208 - FLLJ
[尝试次数：1] - [剩余数量：353] - [当前时间：15:50:56]

[ok] - 515

[ok] - LK001PB - FLLJ
[尝试次数：2] - [剩余数量：234] - [当前时间：15:51:20]

[ok] - 515050 - FLLJ
[尝试次数：1] - [剩余数量：233] - [当前时间：15:51:21]

[ok] - 515046 - FLLJ
[尝试次数：1] - [剩余数量：232] - [当前时间：15:51:21]

[ok] - LK001 - FLLJ
[尝试次数：2] - [剩余数量：231] - [当前时间：15:51:21]

[ok] - LK002PB - FLLJ
[尝试次数：2] - [剩余数量：230] - [当前时间：15:51:21]

[ok] - 515047 - FLLJ
[尝试次数：1] - [剩余数量：229] - [当前时间：15:51:22]

[ok] - LK049 - FLLJ
[尝试次数：25] - [剩余数量：228] - [当前时间：15:51:22]

[ok] - 515073 - FLLJ
[尝试次数：1] - [剩余数量：227] - [当前时间：15:51:22]

[ok] - 513012 - FLLJ
[尝试次数：6] - [剩余数量：226] - [当前时间：15:51:22]

[ok] - 513207 - FLLJ
[尝试次数：2] - [剩余数量：225] - [当前时间：15:51:22]

[ok] - 512152 - FLLJ
[尝试次数：11] - [剩余数量：224] - [当前时间：15:51:22]

[ok] - 512174 - FLLJ
[尝试次数：1] - [剩余数量：223] - [当前时间：15:51:22]

[ok] - 512187 - FLLJ
[尝试次数：2] - [剩余数量：222] - [当前时间：15:51:23]

[ok] - LK002 - FLLJ
[尝试次数：8] - [剩余数量：221] - [当前时间：15:51:23]

[ok] - 512277 - FLLJ
[尝试次数：4] - [剩余数量：220] - [当前时间：15:51:23]

[ok] - 512338 - FLLJ
[尝试次数：1] - [剩余数量：219] - [当前时间：15:51:23]

[ok] - 

[ok] - 512038 - FLLJ
[尝试次数：1] - [剩余数量：101] - [当前时间：15:51:46]

[ok] - 512118 - FLLJ
[尝试次数：1] - [剩余数量：100] - [当前时间：15:51:47]

[ok] - 513035 - FLLJ
[尝试次数：3] - [剩余数量：99] - [当前时间：15:51:47]

[ok] - 512162 - FLLJ
[尝试次数：5] - [剩余数量：98] - [当前时间：15:51:47]

[ok] - 518511 - FLLJ
[尝试次数：1] - [剩余数量：97] - [当前时间：15:51:48]

[ok] - 512165 - FLLJ
[尝试次数：3] - [剩余数量：96] - [当前时间：15:51:48]

[ok] - 512034 - FLLJ
[尝试次数：4] - [剩余数量：95] - [当前时间：15:51:48]

[ok] - 515006 - FLLJ
[尝试次数：1] - [剩余数量：94] - [当前时间：15:51:48]

[ok] - 512013 - FLLJ
[尝试次数：2] - [剩余数量：93] - [当前时间：15:51:48]

[ok] - 512123 - FLLJ
[尝试次数：21] - [剩余数量：92] - [当前时间：15:51:49]

[ok] - 512042 - FLLJ
[尝试次数：1] - [剩余数量：91] - [当前时间：15:51:49]

[ok] - 515028 - FLLJ
[尝试次数：2] - [剩余数量：90] - [当前时间：15:51:49]

[ok] - 515049 - FLLJ
[尝试次数：2] - [剩余数量：89] - [当前时间：15:51:49]

[ok] - 515022 - FLLJ
[尝试次数：11] - [剩余数量：88] - [当前时间：15:51:50]

[ok] - 515069 - FLLJ
[尝试次数：1] - [剩余数量：87] - [当前时间：15:51:50]

[ok] - 515011 - FLLJ
[尝试次数：4] - [剩余数量：86] - [当前时间：15:51:50]

[ok] - 515070 - FLLJ